# HW05: Double ML with XGBoost
(due October 27th)

In [1]:
# Load Data (NLSY)
import pandas as pd
df = pd.read_stata('http://www.stata-press.com/data/r10/nlswork.dta')

In [2]:
df.describe()

,idcode,year,birth_yr,age,race,msp,nev_mar,grade,collgrad,not_smsa,...,south,ind_code,occ_code,union,wks_ue,ttl_exp,tenure,hours,wks_work,ln_wage
count,28534.000000,28534.000000,28534.000000,28510.000000,28534.000000,28518.000000,28518.000000,28532.000000,28534.000000,28526.000000,...,28526.000000,28193.000000,28413.000000,19238.000000,22830.000000,28534.000000,28101.000000,28467.000000,27831.000000,28534.000000
mean,2601.284292,77.958646,48.085091,29.045107,1.303392,0.602917,0.229680,12.532595,0.168045,0.282444,...,0.409556,7.692973,4.777672,0.234432,2.548095,6.215322,3.123830,36.559560,53.989328,1.674888
std,1487.358947,6.383879,3.012837,6.700584,0.482277,0.489302,0.420634,2.323905,0.373913,0.450196,...,0.491761,2.994025,3.065435,0.423654,7.294463,4.652105,3.751425,9.869623,29.032325,0.478094
min,1.000000,68.000000,41.000000,14.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
25%,1327.000000,72.000000,46.000000,23.000000,1.000000,0.000000,0.000000,12.000000,0.000000,0.000000,...,0.000000,5.000000,3.000000,0.000000,0.000000,2.461539,0.500000,35.000000,36.000000,1.361496
50%,2606.000000,78.000000,48.000000,28.000000,1.000000,1.000000,0.000000,12.000000,0.000000,0.000000,...,0.000000,7.000000,3.000000,0.000000,0.000000,5.057693,1.666667,40.000000,52.000000,1.640541
75%,3881.000000,83.000000,51.000000,34.000000,2.000000,1.000000,0.000000,14.000000,0.000000,1.000000,...,1.000000,11.000000,6.000000,0.000000,0.000000,9.128204,4.166667,40.000000,72.000000,1.964083
max,5159.000000,88.000000,54.000000,46.000000,3.000000,1.000000,1.000000,18.000000,1.000000,1.000000,...,1.000000,12.000000,13.000000,1.000000,76.000000,28.884615,25.916666,168.000000,104.000000,5.263916


In [3]:
df = df.dropna()

In [4]:
df.head()

,idcode,year,birth_yr,age,race,msp,nev_mar,grade,collgrad,not_smsa,...,south,ind_code,occ_code,union,wks_ue,ttl_exp,tenure,hours,wks_work,ln_wage
2,1,72,51,20.0,2,1.0,0.0,12.0,0,0.0,...,0.0,4.0,6.0,1.0,0.0,2.256410,0.916667,40.0,51.0,1.589977
5,1,77,51,25.0,2,0.0,0.0,12.0,0,0.0,...,0.0,12.0,8.0,0.0,0.0,3.775641,1.500000,32.0,52.0,1.778681
7,1,80,51,28.0,2,0.0,0.0,12.0,0,0.0,...,0.0,5.0,6.0,1.0,0.0,5.294872,1.833333,45.0,75.0,2.551715
9,1,85,51,33.0,2,0.0,0.0,12.0,0,0.0,...,0.0,5.0,6.0,1.0,0.0,7.160256,1.916667,42.0,97.0,2.614172
10,1,87,51,35.0,2,0.0,0.0,12.0,0,0.0,...,0.0,5.0,6.0,1.0,0.0,8.987180,3.916667,45.0,95.0,2.536374


In [5]:
import numpy as np
Y = df['ln_wage']
D = df['union']

In [6]:
# fill in X with all predictors that are not colliders
X = df[['age', 'year', 'race', 'msp', 'nev_mar', 'grade', 'not_smsa', 'south', 'wks_ue', 'ttl_exp']]

In [7]:
# split into sample A and sample B

from sklearn.model_selection import train_test_split

seed = 349856

a_x, b_x, a_y, b_y, a_d, b_d = train_test_split(X, Y, D, test_size=0.5, random_state=seed)

train_a_x, test_a_x, train_a_y, test_a_y, train_a_d, test_a_d = train_test_split(a_x, a_y, a_d, test_size=0.2, random_state=seed)

train_b_x, test_b_x, train_b_y, test_b_y, train_b_d, test_b_d = train_test_split(b_x, b_y, b_d, test_size=0.2, random_state=seed)

In [8]:
# Step 1. In both samples, use xgboost regressor to predict log wages (outcome Y)
# use early stopping.
from xgboost import XGBRegressor

a_xgbr = XGBRegressor(objective ='reg:squarederror', 
                          colsample_bytree = 0.3, 
                          learning_rate = 0.1,
                          max_depth = 5, 
                          alpha = 10, 
                          n_estimators = 10)
a_xgbr.fit(train_a_x, train_a_y)

b_xgbr = XGBRegressor(objective ='reg:squarederror', 
                          colsample_bytree = 0.3, 
                          learning_rate = 0.1,
                          max_depth = 5, 
                          alpha = 10, 
                          n_estimators = 10)
b_xgbr.fit(train_b_x, train_b_y)

XGBRegressor(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.3, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.1, max_delta_step=0, max_depth=5,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=10, n_jobs=0, num_parallel_tree=1, random_state=0,
             reg_alpha=10, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [9]:
# Step 2. In both samples, use xgboost classifier to predict union status (treatment D)
# use early stopping.
from xgboost import XGBClassifier

a_xgbc = XGBClassifier(random_state = seed, eval_set = [test_a_x, test_a_d], early_stopping_rounds = 10)
a_xgbc.fit(train_a_x, train_a_d)

b_xgbc = XGBClassifier(random_state = seed, eval_set = [test_b_x, test_b_d], early_stopping_rounds = 10)
b_xgbc.fit(train_b_x, train_b_d)

[14:54:36] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { early_stopping_rounds, eval_set } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[14:54:36] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { early_stopping_rounds, eval_set } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, early_stopping_rounds=10,
              eval_set=[        age  year  race  msp  nev_mar  grade  not_smsa  south  wks_ue  \
4110   35.0    78     2  0.0      0.0   10.0       0.0    0.0     0.0   
22947  38.0    87     1  1.0      0.0   10.0       1.0    0.0     0.0   
2038   25.0    70     1  1.0      0.0   10.0       0.0    0.0     0.0   
5484   29.0    72     1  1.0      0.0   13.0       0.0    0.0     0.0   
2413   25.0    73     1  0.0      1.0   12.0       0.0    0.0     0.0   
...     ...   ...   ...  .....
              gamma=0, gpu_id=-1, importance_type='gain',
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=0,
              num_parallel_tree=1, random_state=349856, reg_alpha=0,
              r

In [10]:
# Step 3. Cross-fitting: Form predictions in other sample.

from sklearn.metrics import (precision_score, recall_score, f1_score, 
                             confusion_matrix, accuracy_score, balanced_accuracy_score, roc_curve, auc, roc_auc_score)

# predict wages in sample A using model trained in sample B:
#Y_hat_A = #TODO
pred_a_y = b_xgbr.predict(a_x)

# vice versa:
pred_b_y = a_xgbr.predict(b_x)
    
# predict union status in sample A using model trained in sample B:
pred_a_d = b_xgbc.predict(a_x)
print(accuracy_score(a_d,pred_a_d))
    
# vice versa:
pred_b_d = a_xgbc.predict(b_x)
print(accuracy_score(b_d,pred_b_d))
    

0.7649420160570919
0.75973832887303


In [11]:
# Compute residuals for wages and union status.

# residualized wages in samples A and B:
Y_tilde_A =  a_y - pred_a_y
Y_tilde_B =  b_y - pred_b_y

# residualized union status in samples A and B:
D_tilde_A =  a_d - pred_a_d
D_tilde_B =  b_d - pred_b_d

In [12]:
# as a shortcut, use statsmodels.
# for projects (not homework), we would do this in stata.
!pip install statsmodels

In [13]:
# Step 4. Run OLS regressions and produce DML estimate

import statsmodels.api as sm

model_A = sm.OLS(Y_tilde_A, D_tilde_A)
result_A = model_A.fit(cov_type='HC1')
beta_A = result_A.params
se_A = result_A.bse

model_B = sm.OLS(Y_tilde_B, D_tilde_B)
result_B = model_B.fit(cov_type='HC1')
beta_B = result_B.params
se_B = result_B.bse

beta = .5 * (beta_A + beta_B)
se = .5 * (se_A + se_B)

print('DML Coeff:',beta)
print('DML S.E.:', se)

DML Coeff: union    0.351685
dtype: float64
DML S.E.: union    0.014884
dtype: float64
